In [8]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Directory where your images are located
image_dir = "normal/"  # Replace with the path to your image directory

# Initialize lists to hold images and filenames
image_data = []
image_names = []

# Load and display the first few images
for i, file_name in enumerate(os.listdir(image_dir)):
    if file_name.endswith('.png') and i < 5:  # Load the first 5 PNG images
        image_path = os.path.join(image_dir, file_name)
        
        # Open the image
        image = Image.open(image_path)
        image_data.append(np.array(image))  # Convert to NumPy array if you need to process as an array
        image_names.append(file_name)

# Display the images
plt.figure(figsize=(15, 5))
for i, img_array in enumerate(image_data):
    plt.subplot(1, len(image_data), i + 1)
    plt.imshow(img_array)
    plt.title(image_names[i])
    plt.axis('off')

plt.show()


<Figure size 1500x500 with 0 Axes>